![alt text](https://www.business.unsw.edu.au/style%20library/asb/assets/images/logo-unsw.png)

# How to use Microsoft Video Indexer API


---

# Overview

## What is Microsoft Video Indexer API?

*An* API that lets us interact with Microsoft's video analysis service. The way it works is that we send files (videos) to the API, then a computer owned by Microsoft analyses the video and sends the results back to us. We will be referring to the returned results as 'features'.

## What results can we get from Microsoft Video Indexer?

A full rundown of the capabilites of Microsoft Video Indexer can be found [here](https://azure.microsoft.com/en-au/services/media-services/video-indexer/)

A list of all the feature outputs from this API can be found [here](https://docs.microsoft.com/en-us/azure/media-services/video-indexer/video-indexer-output-json-v2).

For those familiar with using APIs, the Microsoft Video Indexer API reference can be found [here](https://api-portal.videoindexer.ai/docs/services/operations/operations/Create-Brand)

# Before starting

## Prerequisites

1. Be familiar of the purpose and capabilities of Microsoft Video Indexer. See the links above.
2. Have your Account ID and Subscription Key for Video Indexer. (go through **Video Indexer Account Subscription Key** file on moodle)
3. Have some videos on your computer or video urls to send to the API. There is a sample video on moodle you can experiment with (called **sample_video**).
4. Click 'CONNECT' button located at top right of notebook (wait until it changes to 'CONNECTED')

# Dependencies

Before you make your imports, we will be using a custom made package to make it easier to Microsoft Video Indexer's API.

To do this, download **week2_helpers.py** from Moodle and upload it to this Google Colab notebook. 

If you do this successully, there will not be any errors from running the cell below.

In [2]:
from google.colab import files

# Upload your week2_helpers.py file from moodle
files.upload()

Saving week2_helpers.py to week2_helpers.py


{'week2_helpers.py': b'"""\r\nSpecial Utility Functions for MARK5828 Week 2 Tutorial Content. The functions saved here are unnecessary for the students to learn, but they can read if they are curious.\r\nAuthor: James Lin\r\nSpecial Thanks: Daniel-Han Chan\r\n"""\r\n\r\nimport itertools\r\nfrom scipy.linalg.blas import ssyrk\r\nfrom scipy.linalg.lapack import spotrf, sposv\r\nimport numpy as np\r\nimport pandas as pd\r\n\r\nimport urllib.parse\r\nimport requests\r\n\r\ndef convert_to_seconds(time_str):\r\n    h, m, s = time_str.split(":")\r\n    return int(h) * 3600 + int(m) * 60 + round(float(s))\r\n\r\ndef get_source_language_v2(data):\r\n    return data["videos"][0]["insights"]["sourceLanguage"]\r\n\r\ndef get_video_duration_v2(data):\r\n    return data["summarizedInsights"]["duration"]["seconds"]\r\n\r\ndef get_faces_v2(data):\r\n    face_names = []\r\n    face_duration = []\r\n\r\n    for i, face in enumerate(data["summarizedInsights"]["faces"]):\r\n        face_names.append(face[

In [0]:
import json

from week2_helpers import VideoIndexerClientV2, video_indexer_to_df

# Upload test videos

## Upload files

In [4]:
# Upload sample_video.mp4 (or any other video). Note it may take a while to upload videos
test_videos = files.upload()

Saving sample_video.mp4 to sample_video (1).mp4


# API access setup



To access the Microsoft Video Indexer API you need to provide your **subscription_key** and **account_id**.

In [0]:
subscription_key = "d3b146210a7b49d6b845cb83d2cdfd2c" #@param {type:"string"}
account_id = "102f1d6a-8100-4874-9dd9-4460d08c9e05" #@param {type:"string"}

In [0]:
indexer = VideoIndexerClientV2(
  subscription_key=subscription_key,
  location="trial",
  account_id=account_id
)

# Using the API

Using Microsoft Video Indexer has a few differences from Google Cloud Vision from Week 1:
- The data we are dealing with are **videos** instead of **images**. Analysing videos can take a very long time to complete (eg, a 3 minute video can take longer than 10 minutes to analyse). For this reason the way we use Video Indexer is:
  1. Upload the video to Video Indexer
  2. Wait until Video Indexer has finished analysing.
  3. Download the results from Video Indexer.
  
There are several functions for you to use. If you want to upload a video on your computer, you would use 

`indexer.upload_video_local(name, video_path, description, external_id, privacy)`

If you want to download the result for a specific video, you would use

`get_video_index(video_id)`

Let's first upload the sample video from Moodle

In [0]:
# @title What is the name of your video? (Moodle's video is called sample_video.mp4)
video_name = "sample_video (1).mp4" #@param {type:"string"}

In [10]:
upload_result = indexer.upload_video_local(
  name="Sample video",               # Your title of this video
  description="Sample Description",  # Your description about this Video
  video_path=video_name,
  external_id="sample_id",           # Video ID of your choosing
  privacy="Private"                  # "Public" or "Private"
)

Uploading Video: Sample video


There are a few important information to note when you print out the results:

`id` : This corresponds to the video_id needed to get the output results later on.

`processingProgress` : This will tell you whether the video has finished been analysed or not.

You can also go to https://www.videoindexer.ai/ and login with your zID to see if your video is currently being processed.

In [15]:
upload_result

{'accountId': '102f1d6a-8100-4874-9dd9-4460d08c9e05',
 'created': '2019-03-06T04:52:18.7678294+00:00',
 'description': 'Sample Description',
 'durationInSeconds': 0,
 'externalId': 'sample_id',
 'hasSourceVideoFile': True,
 'id': '6fd33e0554',
 'indexingPreset': 'Default',
 'isBase': True,
 'isOwned': True,
 'lastIndexed': '2019-03-06T04:52:18.8147037+00:00',
 'lastModified': '2019-03-06T04:52:18.8147037+00:00',
 'metadata': None,
 'moderationState': 'OK',
 'name': 'Sample video',
 'partition': None,
 'personModelId': '00000000-0000-0000-0000-000000000000',
 'privacyMode': 'Private',
 'processingProgress': '10%',
 'reviewState': 'None',
 'searchMatches': [],
 'sourceLanguage': 'en-US',
 'state': 'Processing',
 'streamingPreset': 'Default',
 'thumbnailId': '00000000-0000-0000-0000-000000000000',
 'thumbnailVideoId': '6fd33e0554',
 'userName': 'Danica Huei Ye Yong'}

Once Video Indexer has finished analysing the video, you can get the results:

The results can be interpreted [here](https://docs.microsoft.com/en-us/azure/media-services/video-indexer/video-indexer-output-json-v2).

In [0]:
# @title When you print upload_result, what is 'id'?
video_id = "6fd33e0554" #@param {type:"string"}

In [17]:
data = indexer.get_video_index(video_id=video_id)
data

{'accountId': '102f1d6a-8100-4874-9dd9-4460d08c9e05',
 'created': '2019-03-06T04:52:18.7678294+00:00',
 'description': 'Sample Description',
 'durationInSeconds': 129,
 'id': '6fd33e0554',
 'isBase': True,
 'isEditable': True,
 'isOwned': True,
 'name': 'Sample video',
 'partition': None,
 'privacyMode': 'Private',
 'state': 'Processed',
 'summarizedInsights': {'audioEffects': [{'appearances': [{'endSeconds': 129.2,
      'endTime': '0:02:09.196',
      'startSeconds': 125.6,
      'startTime': '0:02:05.623'}],
    'audioEffectKey': 'Silence',
    'seenDuration': 3.6,
    'seenDurationRatio': 0.0279}],
  'brands': [],
  'duration': {'seconds': 129.2, 'time': '0:02:09.238'},
  'emotions': [{'appearances': [{'endSeconds': 94.9,
      'endTime': '0:01:34.9',
      'startSeconds': 89.2,
      'startTime': '0:01:29.24'}],
    'seenDurationRatio': 0.0441,
    'type': 'Fear'}],
  'faces': [{'appearances': [{'endSeconds': 116,
      'endTime': '0:01:56.016',
      'startSeconds': 97.2,
      '

# Store results in a file



Microsoft Video Indexer returns a JSON object. We can convert this into a clean dataframe using the **video_indexer_to_df()** function, which simply takes in a list of JSON objects.

Once we have a dataframe, we can save it into a .csv file.

In [18]:
json_list = [data, data, data, data]

df = video_indexer_to_df(json_list)
df

Processing: indexer_audio_effects
Processing: indexer_brands
Processing: indexer_emotions
Processing: indexer_frame_patterns
Processing: indexer_keywords
Processing: indexer_labels
Processing: indexer_ocr
Processing: indexer_topics


,id,indexer_content_moderation_banned_words_count,indexer_content_moderation_banned_words_ratio,indexer_duration,indexer_sentiment_negative,indexer_sentiment_neutral,indexer_sentiment_positive,indexer_source_language,indexer_transcript,indexer_audio_effects_silence,...,indexer_ocr_15_years_old_area,indexer_ocr_damascus_area,indexer_ocr_unicef@television_area,indexer_ocr_myw.unicef.org_area,indexer_ocr_bashar_area,indexer_ocr_unite_for_children_area,indexer_ocr_unicef@_area,indexer_ocr_dedeman_area,indexer_num_faces,indexer_statistics_correspondence_count
0,0,0,0,129.2,6,123,0,en-US,291,4,...,2464,2508,8959,2412,1660,4902,7700,6688,34,2
1,1,0,0,129.2,6,123,0,en-US,291,4,...,2464,2508,8959,2412,1660,4902,7700,6688,34,2
2,2,0,0,129.2,6,123,0,en-US,291,4,...,2464,2508,8959,2412,1660,4902,7700,6688,34,2
3,3,0,0,129.2,6,123,0,en-US,291,4,...,2464,2508,8959,2412,1660,4902,7700,6688,34,2


In [0]:
df.to_csv("my_video_indexer_data.csv", index=False)
files.download('my_video_indexer_data.csv')
